<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch10/section_91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 91. 続きのテキスト予測
“The movie was full of”に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。
### temperature（温度）パラメータとは
- softmax関数に渡す定数（例としてTとしている）
$$
p_i = softmax(\frac{z_i}{T})
$$
### 温度パラメータの変更による影響
- 温度パラメータはGPTが応答を生成する際に、生成される応答の多様性を制御するために調整される。
- 温度パラメータが高いほど、よりランダムな応答が生成される。
- 逆に温度パラメータが低いほど、より確信度の高い応答が生成される。

補足
- 今回はPADとEOSを設定する必要はないが慣習として設定する

In [50]:
!pip install -U transformers

### GenerateConfigとは？
- transformersライブラリに用意されているテキスト生成専用の設定クラス
- model.generate()の引数に設定して、デコーディング時の挙動をまとめて指定するためのクラス


In [51]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [52]:
model_id = "gpt2"
device = "cuda" if torch.cuda.is_available() else "cpu"
temperatures = [1.0, 0.7, 0.5, 0.1]
max_new_tokens = 50

In [53]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [54]:
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

In [56]:
text = "The movie was full of"

# paddingとtruncationは慣習的に設定している
input_ids = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)

model.generate()はtoken_idのtensorを返す
## Greedy methodとは？
- 毎ステップ一番確率が高い次トークンを一つだけ選び続けるデコーディング方法

In [57]:
print("Greedy method")
with torch.no_grad():
  generation_config = GenerationConfig(
      max_new_tokens=max_new_tokens,
      pad_token_id=tokenizer.pad_token_id,
      eos_token_id=tokenizer.eos_token_id,
      do_sample=False, # beamが指定されていない場合はGreedy
  )
  output = model.generate(**input_ids, generation_config=generation_config)
  sampling_output = tokenizer.decode(output[0], skip_special_tokens=True)
  print(sampling_output)

Greedy method
The movie was full of jokes and jokes about how the movie was a joke. It was a joke about how the movie was a joke. It was a joke about how the movie was a joke. It was a joke about how the movie was a joke. It was a


## BeamSearch（ビームサーチ）とは？
- 生成は1トークンずつ進む。各ステップで候補（ビーム）を展開し、累積スコア（通常はlog確率の和）が高い上位 beam_size 本だけを残す（剪定する）。
- これを max_new_tokens 回（または EOS が出るまで）繰り返し、最終的に最もスコアの高い系列（ルート）を出力する。


In [58]:
print("BeamSearch（num_beams = 5）")
with torch.no_grad():
  generation_config = GenerationConfig(
      max_new_tokens=max_new_tokens,
      pad_token_id=tokenizer.pad_token_id,
      eos_token_id=tokenizer.eos_token_id,
      do_sample=False,
      num_beams=5
  )
  output = model.generate(**input_ids, generation_config=generation_config)
  sampling_output = tokenizer.decode(output[0], skip_special_tokens=True)
  print(sampling_output)

BeamSearch（num_beams = 5）
The movie was full of jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and


In [59]:
print("Multinominal Sampling")
for t in temperatures:
  print(f"Tempreture = {t}")
  with torch.no_grad():
    generation_config = GenerationConfig(
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        num_beams=1,
        temperature=t,
    )
    output = model.generate(**input_ids, generation_config=generation_config)
    sampling_output = tokenizer.decode(output[0], skip_special_tokens=True)
    print(sampling_output)

Multinominal Sampling
Tempreture = 1.0
The movie was full of jokes about how the show is "totally not real." The cast was also shocked with it, and it certainly wasn't just about the kids growing up in suburban Detroit.

On the plus side, James and his co-workers were making
Tempreture = 0.7
The movie was full of references to the 1970s, but it was also full of references to how the world of the film would react in the years to come. It's one of those movies that's full of great ideas, but I can't think of one in the
Tempreture = 0.5
The movie was full of great action. The film is about a group of young girls who are given a choice between a life of crime and a life of love. The film is about a group of young girls who are given a choice between a life of crime and a life
Tempreture = 0.1
The movie was full of great moments, but it was also full of bad moments.

I'm not sure if it's because I'm a fan of the movie or if it's because I'm a fan of the characters. I'm not sure if it's beca